# Brasileirão Série A: O caminho dos campeões
Por [Daniel Soares](https://www.linkedin.com/in/daniel-soares-ti/)


<div style="display:inline-block;vertical-align:top;">
<h2>Sumário:</h2>
<ul>
    <li><a href="#introducao">Introdução </a></li>
    <li><a href="#maior-campeao">Maiores campeões</a></li>
    <li><a href="#responsabilidade">Melhor e "pior" campeão</a></li>
    <li><a href="#tecnicas_metodos">Ano mais competitivo</a></li>
    <li><a href="#tecnicas_metodos">Ano mais tranquilo</a></li>
    <li><a href="#tecnicas_metodos">Dominância? Houve anos consecutivos, times que dominaram certo período?</a></li>

</ul>
</div>
<div style="display:inline-block; margin-left:50px; margin-top:15px">
<img src="https://media.giphy.com/media/xUOxf1JRLpx5wy7UyY/giphy.gif" style="height: 250px; width:400px;"/>
    <div style="text-align:center; font-size:12px"><i>Fonte: <a href="https://media.giphy.com/media/xUOxf1JRLpx5wy7UyY/giphy.gif">Giphy</a></i></div>
</div>

--------


In [1]:
import pandas as pd
import plotly.express as px

# Lendo os dados
games_data = pd.read_csv('../data/campeonato-brasileiro-full.csv')

In [ ]:
def corrigir_indexes_e_colunas_tabela(dataframe):
    """Corrige indexes e colunas do dataframe

    Args:
        dataframe (pd.Dataframe): Dataframe com os jogos

    Returns:
        pd.Dataframe: Dataframe com os indexes e colunas corrigidos
    """
    dataframe = dataframe.reset_index()
    dataframe = dataframe.set_index(pd.Index(range(1, 21)))
    dataframe = dataframe.rename(columns={'vencedor': 'times', 'count': 'pontos'})
    
    return dataframe

def calcular_pontos_de_vitoria(dataframe_original, ano):
    """Calcula a pontuação inicial dos times com base em suas vitórias

    Args:
        dataframe_original (pd.Dataframe): Dataframe com os jogos
        ano (int): ano do campeonato

    Returns:
        pd.Dataframe: Dataframe com os pontos iniciais calculados
    """
    tabela_campeonato = dataframe_original.query(f'ano == {ano}').vencedor\
        .value_counts()[1:].to_frame()
    tabela_campeonato = tabela_campeonato * 3
    
    return tabela_campeonato

def adicionar_pontos_empate(dataframe_original, dataframe_pontos, ano):
    """Adiciona pontos para os jogos que terminaram empatados

    Args:
        dataframe (pd.Dataframe): dataframe com os jogos
        dataframe_pontos (pd.Dataframe): dataframe com os pontos iniciais
        ano (int): ano do campeonato

    Returns:
        pd.Dataframe: dataframe com os pontos de empate inseridos
    """
    for time in dataframe_original.query(f"ano == {ano} and vencedor =='-'")\
        .mandante:
        dataframe_pontos.loc[time] = dataframe_pontos.loc[time] + 1
        
    for time in dataframe_original.query(f"ano == {ano} and vencedor =='-'")\
        .visitante:
        dataframe_pontos.loc[time] = dataframe_pontos.loc[time] + 1
    
    return dataframe_pontos

<a id='introducao'></a>
# Introdução
"O Brasileirão é o campeonato mais difícil do mundo". Essa é uma frase que você 
provavelmente já escutou na rua, e que vem provando ser verdadeira. A Federação 
Internacional de História e Estatísticas do Futebol (IFFHS), organização 
reconhecida pela FIFA, [publicou](https://www.iffhs.com/posts/2483) no início de 
2023 um ranking de campeonatos nacionais, e levando em conta fatores como força 
dos concorrentes e conquistas dos times na temporada, coroou o Brasileirão como 
o mais difícil do mundo pelo segundo ano consecutivo.

Diante disso, chegamos a uma pergunta: Qual o caminho dos campeões? O que um time
precisa para conquistar o atual campeonato mais difícil do planeta?

<div class="alert alert-warning" role="alert">
    <b>Aviso!</b> Os campeonatos de 2003, 2004 e 2005, por possuírem um número 
    maior de rodadas, serão desconsiderados nesta análise. Entretanto, tais 
    campeonatos poderão ser citados.
</div>

In [3]:
# Para evitar problemas quando for plotar os gráficos, vou alterar o dtype
# das colunas que necessitam
games_data['data'] = pd.to_datetime(games_data['data'], dayfirst=True)
games_data['hora'] = pd.to_datetime(games_data['hora'], format='%H:%M').dt.time

# Para separar os campeonatos por ano, irei criar uma coluna com o ano 
# de cada jogo
games_data['ano'] = games_data['data'].dt.year

In [4]:
# Verificando se a quantidade de rodadas está correta
games_data.groupby('ano').rodata.nunique()

ano
2003    46
2004    46
2005    42
2006    38
2007    38
2008    38
2009    38
2010    38
2011    38
2012    38
2013    38
2014    38
2015    38
2016    38
2017    38
2018    38
2019    38
2020    27
2021    38
2022    38
Name: rodata, dtype: int64

In [25]:
# Para gerar as classificações de forma correta, terei que separar os anos
# que tiveram punições e os que não tiveram para então usar as funções
anos_sem_punicao = [2006, 2007, 2008, 2009, 2011, 2012, 2014, 2015, 2017, 2019, 
                    2020, 2021, 2022]

anos_com_punicao = [2010, 2013, 2018]

classificacao = {}

for ano in anos_sem_punicao:
    classificacao[ano] = calcular_pontos_de_vitoria(games_data,
                                                     ano)
    
    classificacao[ano] = adicionar_pontos_empate(games_data, 
                                                  classificacao[ano],
                                                  ano)
    
    classificacao[ano] = classificacao[ano].sort_values(by='count',
                                                          ascending=False)
    
    #classificacao[ano] = corrigir_indexes_e_colunas_tabela(classificacao[ano])
    
    

In [ ]:
# Reordenar o ano de 2017 para corrigir os clubes rebaixados
# O ano de 2020 possui rodadas em 2021

<a id='maior-campeao'></a>
# Maiores campeões


In [6]:
tabela_2006 = calcular_pontos_de_vitoria(games_data, 2006)
tabela_2006 = adicionar_pontos_empate(games_data, tabela_2006, 2006)
tabela_2006 = tabela_2006.sort_values(by='count', ascending=False)
tabela_2006 = corrigir_indexes_e_colunas_tabela(tabela_2006)